<a href="https://colab.research.google.com/github/reykel/cat_replaced_by_dog/blob/main/Application_for_recruiter_cat_replaced_by_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install torch torchvision opencv-python pillow numpy

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import numpy as np
import cv2

def segment_image(cat_image, class_index):
    img = Image.open(cat_image).convert('RGB')

    preprocess = transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    input_image = preprocess(img).unsqueeze(0)

    with torch.no_grad():
        output = model(input_image)['out'][0]

    output_predictions = output.argmax(0)

    cat_mask = output_predictions == class_index

    if cat_mask.sum() > 0:
        return cat_mask.numpy()
    else:
        return None

def remove_cat_and_add_dog(cat_image, dog_image):
    cat_mask = segment_image(cat_image, 8)
    if cat_mask is None:
        print("No cat detected")
        return

    img = cv2.imread(cat_image)

    cat_mask_resized = cv2.resize(cat_mask.astype(np.uint8), (img.shape[1], img.shape[0]))

    inpainted_img = cv2.inpaint(img, cat_mask_resized, inpaintRadius=7, flags=cv2.INPAINT_TELEA)

    dog_mask = segment_image(dog_image, 12)
    if dog_mask is None:
        print("No dog detected")
        return

    dog_img = cv2.imread(dog_image)

    dog_img_resized = cv2.resize(dog_img, (img.shape[1], img.shape[0]))
    dog_mask_resized = cv2.resize(dog_mask.astype(np.uint8), (img.shape[1], img.shape[0]))

    inpainted_img[dog_mask_resized > 0] = dog_img_resized[dog_mask_resized > 0]

    cv2.imwrite('final_image.jpg', inpainted_img)

    return inpainted_img

def segment_and_replace_cat_with_dog(cat_image, dog_image):
    result_img = remove_cat_and_add_dog(cat_image, dog_image)
    if result_img is not None:
        print("Successfully replaced")
    else:
        print("Operation failed")


model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet101', pretrained=True)
model.eval()

cat_image = 'cat.jpg'
dog_image = 'dog.jpg'

segment_and_replace_cat_with_dog(cat_image, dog_image)


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


Successfully replaced
